## 파이썬으로 PDF 파일 원하는 대로 추출하기


샘플문서 : 네이버 2023 사업보고서 (https://www.navercorp.com/investment/irReport)

In [1]:
## 필요 모듈 다운로드
!pip3 install pymupdf


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip


In [2]:
import pymupdf  
from typing import List, Tuple, Dict

In [3]:
# pdf 파일을 열어보기
pdf_path = "네이버_사업보고서_2023.pdf"

doc = pymupdf.open(pdf_path)
print(doc)

Document('네이버_사업보고서_2023.pdf')


In [4]:
## document는 각각의 페이지로 구성된 리스트

## 아래를 보면 페이지 수랑 일치함
print(len(doc))

## 첫페이지는 이렇게 접근 가능함
print(doc[0])  

565
page 0 of 네이버_사업보고서_2023.pdf


In [5]:
## 페이지의 정보를 dict 형태로 반환
doc[0].get_text("dict")

## 기본 페이지정보 (너비(width),높이(height) 너비, 블럭) 등이 있음
## 1개의 블럭은 1개의 사각형 영역을 의미


{'width': 595.0,
 'height': 842.0,
 'blocks': [{'number': 0,
   'type': 0,
   'bbox': (236.2100067138672, 60.265625, 358.7860412597656, 76.265625),
   'lines': [{'spans': [{'size': 16.0,
       'flags': 4,
       'font': 'Batang',
       'color': 0,
       'ascender': 0.8583984375,
       'descender': -0.1416015625,
       'text': '목                 차',
       'origin': (236.2100067138672, 74.0),
       'bbox': (236.2100067138672, 60.265625, 358.7860412597656, 76.265625)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (236.2100067138672, 60.265625, 358.7860412597656, 76.265625)}]},
  {'number': 1,
   'type': 0,
   'bbox': (66.0, 89.416015625, 545.0070190429688, 99.416015625),
   'lines': [{'spans': [{'size': 10.0,
       'flags': 4,
       'font': 'Batang',
       'color': 0,
       'ascender': 0.8583984375,
       'descender': -0.1416015625,
       'text': '사 업 보 고 서...............................................................................................................

In [6]:
## 기본적으로 1개의 페이지는 여러 블럭으로 나눠어져 있음

## 회사의 개요 페이지는 33개 블럭으로 되어있네
len(doc[5].get_text("dict")["blocks"])

33

In [7]:
## 블럭정보에는 
# - number : 블럭번호, 0이면 페이지의 첫번째 블럭)
# - type : 블럭타입, 0이면 텍스트, 1이면 이미지
# - bbox : 블럭의 경계좌표
# - lines : 여러 줄로 이루어진 문단일 경우 여러 개의 줄이 포함된 리스트
# 블럭 1개를 자세히 보면 각각의 줄 (line)로 이루어져 있음

## 1개의 줄에 그 줄의 텍스트 정보가 나와있음
# - text : 실제로 우리가 관심이 있는 텍스트
# - font : 글꼴
# - size : 글자크기

doc[5].get_text("dict")["blocks"][0]

{'number': 0,
 'type': 0,
 'bbox': (240.9499969482422, 61.548828125, 354.0439758300781, 79.548828125),
 'lines': [{'spans': [{'size': 18.0,
     'flags': 4,
     'font': 'Batang',
     'color': 255,
     'ascender': 0.8583984375,
     'descender': -0.1416015625,
     'text': 'I. 회사의 개요',
     'origin': (240.9499969482422, 77.0),
     'bbox': (240.9499969482422,
      61.548828125,
      354.0439758300781,
      79.548828125)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (240.9499969482422,
    61.548828125,
    354.0439758300781,
    79.548828125)}]}

In [10]:
## 그래서 우리는 pdf 문서안의 모든 페이지, 모든 블럭, 모든 줄을 돌면서
## 모든 텍스트와 폰트크기를 같이 구해서 리스트로 만들어줄 것임

## 일단 모든 텍스트를 텍스트 / 폰트크기 조합으로 추출해보기
TextBlock = Tuple[str, float]

def extract_texts_with_font_size(pdf_path: str) -> List[TextBlock]:
    doc = pymupdf.open(pdf_path)
    content = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            lines = block.get("lines", [])

            for line in lines:
                for span in line["spans"]:
                    text = span["text"]
                    size = span["size"]
                    content.append((text, size))
    
    return content
result = extract_texts_with_font_size(pdf_path)


In [11]:
for i in result[1000:1100]:
  print(i)

('5. 정관에 관한 사항', 16.0)
(' ', 12.0)
('가. 정관변경 이력', 12.0)
(' ', 12.0)
('당사는 2021년 3월 24일에 개최한 제22기 정기주주총회에서 정관이 최신 개정되었으며, 공', 12.0)
('시대상기간(2021년~2023년)중 정관 변경 이력은 다음과 같습니다.', 12.0)
(' ', 12.0)
(' ', 12.0)
('나. 사업목적 현황', 12.0)
(' ', 12.0)
('정관변경일', 8.0)
('해당주총명', 8.0)
('주요변경사항', 8.0)
('변경이유', 8.0)
('2021년 03월 24일', 8.0)
('제22기 정기주주총회', 8.0)
('제3조(본점의소재지및 지점 등의 설치)', 8.0)
('제15조(명의개서대리인)', 8.0)
('제16조(주주명부 작성 비치)', 8.0)
('제17조(주주명부의 폐쇄 및 기준일)', 8.0)
('제37조(이사의 임기)', 8.0)
('부칙', 8.0)
('- 본점 소재지 최소 행정 단위로 수정', 8.0)
('- 전자등록제도 도입에 따른 변동사항 반영', 8.0)
('- 관련 법규에 의한 근거규정 신설 등', 8.0)
('- 전자등록제도 도입에 따른 변동사항 반영', 8.0)
('- 관련 상법규정 준용', 8.0)
('- 정관 시행일자 추가', 8.0)
('※ 당사는 공시서류 제출일 이후인 2024년 3월 26일에 제25기 정기주주총회를 개최할 예정이며, 해당 주주총회 안건으로', 9.0)
('정관변경', 9.0)
("    안건이 상정되어 있습니다. 안건의 세부 내용은 당사가 2024년 2월 26일에 공시한 '주주총회소집공고' 공시를 참조해", 9.0)
('주시기', 9.0)
('    바라며, 안건이 부결되거나 수정이 발생할 경우 정정보고서를 통해 그 내용 및 사유 등을 반영할 예정 입니다.', 9.0)
('구 분', 10.0)
('사업목적', 10.0)
('사업영위 여부', 10.0)
('1', 10.0)
('컴퓨터 및 통신기기를 이용한 정보자료처리

In [17]:
## 소제목이 되는 텍스트의 글자 크기를 확인해보기 => 16인걸 확인했음!

some_titles = ㄴ
title_font_sizes = []

for text, size in result:
        if text in some_titles:
            title_font_sizes.append((text, size))
print(title_font_sizes)

[('1. 사업의 개요', 16.0), ('2. 주요 제품 및 서비스', 16.0)]


In [18]:
## 특정 글자 크기 기준으로 텍스트들을 쪼개는 함수

def create_chunks_based_on_font_size(content: List[TextBlock], target_font_size: float) -> List[List[TextBlock]]:
    chunks = []  # 텍스트 블록의 청크를 저장할 리스트
    current_chunk = []  # 현재 청크를 저장할 임시 리스트

    for text_block in content:
        text, font_size = text_block

        if font_size == target_font_size:
            # 대상 글꼴 크기를 가진 텍스트 블록을 찾았습니다.
            # 이는 새로운 청크의 시작을 의미합니다.

            # 현재 청크에 텍스트 블록이 있으면 저장합니다.
            if current_chunk:
                chunks.append(current_chunk)
            
            # 이 텍스트 블록으로 새로운 청크를 시작합니다.
            current_chunk = [text_block]
        else:
            # 이 텍스트 블록은 대상 글꼴 크기를 가지고 있지 않습니다.
            # 현재 청크에 추가합니다.
            current_chunk.append(text_block)

    # 마지막 청크가 비어 있지 않으면 추가합니다.
    if current_chunk:
        chunks.append(current_chunk)

    return chunks


In [20]:

# 테스트용 데이터
content = [
    ("제목 1", 12.0),
    ("내용 1 아래 내용입니다.", 10.0),
    ("제목 2", 12.0),
    ("제목 2 아래 내용입니다.", 10.0),
    ("제목 2 아래 추가 내용입니다.", 10.0),
    ("제목 3", 12.0),
    ("제목 3 아래 내용입니다.", 10.0)
]
target_font_size = 12.0
chunks = create_chunks_based_on_font_size(content, target_font_size)
for i, chunk in enumerate(chunks):
    print(f"\n 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")


 청크 1:
  제목 1 (12.0)
  내용 1 아래 내용입니다. (10.0)

 청크 2:
  제목 2 (12.0)
  제목 2 아래 내용입니다. (10.0)
  제목 2 아래 추가 내용입니다. (10.0)

 청크 3:
  제목 3 (12.0)
  제목 3 아래 내용입니다. (10.0)


In [21]:
## 그대로 사업보고서 데이터에 적용해보기 -> 44개의 덩어리들이 만들어짐

chunks = create_chunks_based_on_font_size(result, 16)
len(chunks)

44

In [26]:

## 이 중에서 특정 제목 기준으로 필터링하기


def filter_chunks_by_text_list(chunks: List[List[TextBlock]], delimiter_texts: List[str]) -> List[List[TextBlock]]:
    filtered_chunks = []
    
    for chunk in chunks:
        # 청크의 첫 번째 텍스트가 구분자 텍스트 목록의 어느 하나와 일치하는지 확인
        if chunk and chunk[0][0] in delimiter_texts:
            filtered_chunks.append(chunk)
    
    return filtered_chunks

# 특정 텍스트 목록을 포함하는 청크를 필터링합니다.
delimiter_texts = ["1. 사업의 개요", "2. 주요 제품 및 서비스"]
filtered_chunks = filter_chunks_by_text_list(chunks, delimiter_texts)

# 필터링된 청크를 출력합니다.
for i, chunk in enumerate(filtered_chunks):
    print(f"필터링된 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

필터링된 청크 1:
  1. 사업의 개요 (16.0)
    (12.0)
  '팀네이버'는 대한민국의 대표적인 IT 테크 기업으로 끊임없는 도전과 성장을 이뤄가고 있습 (12.0)
  니다. 첨단의 기술을 일상의 서비스에 담아 사용자에게 새로운 연결의 경험을 선보이는 도전 (12.0)
  을 멈추지 않음으로써, 다양한 기회와 가능성을 열어 나가고 네이버를 둘러싼 모든 이해관계 (12.0)
  자들에게 차별화된 가치를 제공하고 있습니다. (12.0)
    (12.0)
  네이버를 둘러싼 환경은 빠르게 변화 중이며, 이러한 변화를 새로운 도약의 기회로 삼아, 그 (12.0)
  동안 회사의 성장을 견인해 왔던 혁신을 바탕으로 글로벌 시장과 신규사업에 대한 도전을 이 (12.0)
  어가고 있습니다. 또한, 네이버는 지속적인 성장을 목표로 기술, 서비스 등에 대한 선제적 투 (12.0)
  자를 진행하며, 핵심 사업의 경쟁력을 끊임없이 강화해 나가고 있습니다. (12.0)
    (12.0)
  네이버는 국내 1위 인터넷 검색 포털 '네이버(NAVER)'를 기반으로 광고, 커머스 사업을 통 (12.0)
  해 매출을 창출하고 있습니다. 아울러 금융 씬파일러(Thin Filer)들을 위한 핀테크, 웹툰, 스 (12.0)
  노우 등의 콘텐츠 서비스, 기업용 솔루션을 제공하는 클라우드 등 다각화된 사업 포트폴리오 (12.0)
  를 기반으로 안정적인 성장을 이어가고 있습니다. 네이버는 기존 사업에서 탄탄한 성장세를 (12.0)
  유지하는 한편, 신성장동력인 커머스, 핀테크, 콘텐츠, 클라우드 사업 모두에서 고르고 꾸준 (12.0)
  한 성장을 이어가고 있습니다. (12.0)
    (12.0)
  당사의 영업부문은 단일 영업부문으로 구성되어 있으나, 최고영업의사결정자에게 보고되는 (12.0)
  서비스별 영업현황은 다음과 같으며, 각 서비스별 영업수익은 다음과 같습니다. 이와 관련한 (12.0)
  상세한 내용은 'II. 사업의 내용 - 2. 주요제품 및 서비스'

In [27]:
## 주석처리된 특정 텍스트들은 다시한번 지워주기
## 보통 주석은 같은 글자 크기인 경우가 많음
##   전자공시시스템 dart.fss.or.kr (9.0)

def exclude_chunks_based_on_font_size(chunks: List[List[TextBlock]], exclude_font_size: float) -> List[List[TextBlock]]:
    """특정 글꼴 크기에 따라 청크를 제외합니다."""
    filtered_chunks = []
    
    for chunk in chunks:
        temp_chunk = []
        for text, size in chunk:
            # exclude_font_size와 일치하는 글꼴 크기를 가진 텍스트 블록을 건너뜁니다.
            if size == exclude_font_size:
                continue
            temp_chunk.append((text, size))
        
        # 임시 청크가 비어 있지 않으면 필터링된 청크에 추가합니다.
        if temp_chunk:
            filtered_chunks.append(temp_chunk)
    
    return filtered_chunks


filtered_chunks_2 = exclude_chunks_based_on_font_size(filtered_chunks,9)

# 필터링된 청크를 출력합니다.
for i, chunk in enumerate(filtered_chunks_2):
    print(f"필터링된 청크 {i+1}:")
    for text, size in chunk:
        print(f"  {text} ({size})")

필터링된 청크 1:
  1. 사업의 개요 (16.0)
    (12.0)
  '팀네이버'는 대한민국의 대표적인 IT 테크 기업으로 끊임없는 도전과 성장을 이뤄가고 있습 (12.0)
  니다. 첨단의 기술을 일상의 서비스에 담아 사용자에게 새로운 연결의 경험을 선보이는 도전 (12.0)
  을 멈추지 않음으로써, 다양한 기회와 가능성을 열어 나가고 네이버를 둘러싼 모든 이해관계 (12.0)
  자들에게 차별화된 가치를 제공하고 있습니다. (12.0)
    (12.0)
  네이버를 둘러싼 환경은 빠르게 변화 중이며, 이러한 변화를 새로운 도약의 기회로 삼아, 그 (12.0)
  동안 회사의 성장을 견인해 왔던 혁신을 바탕으로 글로벌 시장과 신규사업에 대한 도전을 이 (12.0)
  어가고 있습니다. 또한, 네이버는 지속적인 성장을 목표로 기술, 서비스 등에 대한 선제적 투 (12.0)
  자를 진행하며, 핵심 사업의 경쟁력을 끊임없이 강화해 나가고 있습니다. (12.0)
    (12.0)
  네이버는 국내 1위 인터넷 검색 포털 '네이버(NAVER)'를 기반으로 광고, 커머스 사업을 통 (12.0)
  해 매출을 창출하고 있습니다. 아울러 금융 씬파일러(Thin Filer)들을 위한 핀테크, 웹툰, 스 (12.0)
  노우 등의 콘텐츠 서비스, 기업용 솔루션을 제공하는 클라우드 등 다각화된 사업 포트폴리오 (12.0)
  를 기반으로 안정적인 성장을 이어가고 있습니다. 네이버는 기존 사업에서 탄탄한 성장세를 (12.0)
  유지하는 한편, 신성장동력인 커머스, 핀테크, 콘텐츠, 클라우드 사업 모두에서 고르고 꾸준 (12.0)
  한 성장을 이어가고 있습니다. (12.0)
    (12.0)
  당사의 영업부문은 단일 영업부문으로 구성되어 있으나, 최고영업의사결정자에게 보고되는 (12.0)
  서비스별 영업현황은 다음과 같으며, 각 서비스별 영업수익은 다음과 같습니다. 이와 관련한 (12.0)
  상세한 내용은 'II. 사업의 내용 - 2. 주요제품 및 서비스'

In [31]:
## 최종 결과 한번 인쇄해보기

concatenated_texts = []
    
for chunk in filtered_chunks_2:
    concatenated_text = "".join(text for text, _ in chunk)
    concatenated_texts.append(concatenated_text)

for text in concatenated_texts:
    print(text)    

1. 사업의 개요 '팀네이버'는 대한민국의 대표적인 IT 테크 기업으로 끊임없는 도전과 성장을 이뤄가고 있습니다. 첨단의 기술을 일상의 서비스에 담아 사용자에게 새로운 연결의 경험을 선보이는 도전을 멈추지 않음으로써, 다양한 기회와 가능성을 열어 나가고 네이버를 둘러싼 모든 이해관계자들에게 차별화된 가치를 제공하고 있습니다. 네이버를 둘러싼 환경은 빠르게 변화 중이며, 이러한 변화를 새로운 도약의 기회로 삼아, 그동안 회사의 성장을 견인해 왔던 혁신을 바탕으로 글로벌 시장과 신규사업에 대한 도전을 이어가고 있습니다. 또한, 네이버는 지속적인 성장을 목표로 기술, 서비스 등에 대한 선제적 투자를 진행하며, 핵심 사업의 경쟁력을 끊임없이 강화해 나가고 있습니다. 네이버는 국내 1위 인터넷 검색 포털 '네이버(NAVER)'를 기반으로 광고, 커머스 사업을 통해 매출을 창출하고 있습니다. 아울러 금융 씬파일러(Thin Filer)들을 위한 핀테크, 웹툰, 스노우 등의 콘텐츠 서비스, 기업용 솔루션을 제공하는 클라우드 등 다각화된 사업 포트폴리오를 기반으로 안정적인 성장을 이어가고 있습니다. 네이버는 기존 사업에서 탄탄한 성장세를유지하는 한편, 신성장동력인 커머스, 핀테크, 콘텐츠, 클라우드 사업 모두에서 고르고 꾸준한 성장을 이어가고 있습니다. 당사의 영업부문은 단일 영업부문으로 구성되어 있으나, 최고영업의사결정자에게 보고되는서비스별 영업현황은 다음과 같으며, 각 서비스별 영업수익은 다음과 같습니다. 이와 관련한상세한 내용은 'II. 사업의 내용 - 2. 주요제품 및 서비스' 와 'II. 사업의 내용-4. 매출 및 수주상황을 참고해 주시기 바랍니다.  - 주요 서비스별 영업현황  - 서비스별 영업수익 서비스주요영업서치플랫폼검색, 디스플레이 등커머스커머스 광고, 중개 및 판매, 멤버십핀테크페이, 플랫폼 서비스 등콘텐츠웹툰, SNOW 등클라우드NCP, 웍스, 클로바 등(누적)(단위 : 백만원,%)구분연결당기전기금액비중금액비중영업수익9,670,644100.08,220,07